### Day 23

In [1]:
import numpy as np
from collections import defaultdict, Counter
from itertools import product

In [2]:
day_i = 23

In [3]:
%run start_day.py $day_i

Initializing day 23


In [4]:
cd /home/vincent/Documents/AdventOfCode/2021

/home/vincent/Documents/AdventOfCode/2021


In [3]:
PATH = f"day{day_i}/input{day_i}"

In [4]:
!wc -l $PATH

5 day23/input23


In [5]:
!head $PATH

#############
#...........#
###A#C#C#D###
  #B#D#A#B#
  #########


In [8]:
!tail $PATH

#############
#...........#
###A#C#C#D###
  #B#D#A#B#
  #########


In [15]:
SOLUTION = "D" * 12 + "C" * 9 + "A" * 10 + "B" * 15 + "A" * 6
moves = [x.strip() for x in SOLUTION]
cost = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}
sum(cost[x] for x in moves)

13066

In [6]:
NEW_INPUT = """#############
#...........#
###A#C#C#D###
  #D#C#B#A#
  #D#B#A#C#
  #B#D#A#B#
  #########
"""

In [7]:
SIMPLE_INPUT = """#############
#...........#
###A#C#B#D###
  #########
"""

# 460

In [8]:
letters = ["A", "B", "C", "D"]

def parse_input(inp):
    positions = {}
    for i, line in enumerate(inp):
        if '.' in line or any(l in line for l in letters):
            for j, x in enumerate(line):
                if x in letters or x == '.':
                    positions[(i, j)] = x
    return positions

In [9]:
# real input
with open(PATH, 'r') as f:
    inputs = parse_input([x for x in f.readlines()])


In [10]:
# new input

inputs_new = parse_input(NEW_INPUT.split('\n'))
inputs_simple = parse_input(SIMPLE_INPUT.split('\n'))

In [11]:
inputs

{(1, 1): '.',
 (1, 2): '.',
 (1, 3): '.',
 (1, 4): '.',
 (1, 5): '.',
 (1, 6): '.',
 (1, 7): '.',
 (1, 8): '.',
 (1, 9): '.',
 (1, 10): '.',
 (1, 11): '.',
 (2, 3): 'A',
 (2, 5): 'C',
 (2, 7): 'C',
 (2, 9): 'D',
 (3, 3): 'B',
 (3, 5): 'D',
 (3, 7): 'A',
 (3, 9): 'B'}

In [42]:
!head $PATH

#############
#...........#
###A#C#C#D###
  #B#D#A#B#
  #########


In [12]:
ROOM_j = {"A": 3, "B": 5, "C": 7, "D": 9}
COST = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

def print_board(d):
    n = max(i for i, _ in d)
    m = max(j for _, j in d)
    for i in range(n+2):
        s = ''.join(d.get((i, j), '#') for j in range(m+2))
        print(s)
        
def neighbors(point):
    i, j = point
    return [(i+1, j), (i, j+1), (i-1, j), (i, j-1)]
        
def can_move(amph, d):
    """
    returns 
     - the list of points that amph can move to
     - whether the amph is going to its room
    """
    if d[amph] not in ROOM_j: return [], False
    i, j = amph
    m = max(j for _, j in d)
    target_j = ROOM_j[d[amph]]
    # amph needs space to move
    if all(d.get((ni, nj), '#') != '.' for ni, nj in neighbors(amph)):
        return [], False
    # if amph is all set in its room, no moving allowed
    if j == target_j and all(d.get((xi, xj), '#') in (d[amph], '#') for xi, xj in d if xi >= 2 and xj == j):
        return [], False
    # amph can only leave the hallway to go to its room
    # if its room is empty / has its friends
    if i == 1:  # amph is in the hallway
        room_free = all(d[(xi, xj)] in ('.', d[amph]) for xi, xj in d if xj == target_j and xi >= 2)
        path_free = all(d[(i, xj)] == '.' for xj in range(target_j, j, int(np.sign(j-target_j))))
        if room_free and path_free:
            return [(max(xi for xi, xj in d if xj == target_j and d[(xi, xj)] == '.'), target_j)], True
        else:
            return [], False
    # amph is in its room and has space to move
    # it can move if there is at least one free spot on either side of its door
    if d[(1, j+1)] == '.' or d[(1, j-1)] == '.':
        options = []
        ej = j-1
        # explore left
        while d.get((1, ej), '#') == '.':
            if ej not in ROOM_j.values():
                options.append((1, ej))
            ej -= 1
        ej = j+1
        # explore right
        while d.get((1, ej), '#') == '.':
            if ej not in ROOM_j.values():
                options.append((1, ej))
            ej += 1
        return sorted(options), False
    return [], False
        

def all_options(d, verbose=False):
    """
    Returns only one option if we can bring an amph home!
    """
    res = {}
    for amph in d:
        options, HOME = can_move(amph, d)
        if options != []:
            res[amph] = options
            if verbose: print(f"{amph} can move --> {options}" + (" HOME!" if HOME else ""))
            if HOME:
                return {amph: options}, True
    if verbose:
        print()
        print_board(d)
    return res, False

class Exploration2:
    def __init__(self, board_dict, max_cost=np.inf, debug=False):
        self.MIN_COST = max_cost
        self.explore(board_dict, debug)
        print(self.MIN_COST)
    
    def explore(self, board_dict, debug=False, score_so_far=0):
        if score_so_far >= self.MIN_COST:
            return
        if debug: 
            print_board(board_dict)
            print()
        total_amph = len([1 for k in board_dict if board_dict[k] != '.'])
        n_home = len([1 for i, j in board_dict if board_dict[(i, j)] != '.' and j == ROOM_j[board_dict[(i,j)]]])
        if n_home == total_amph:
            if debug: print(f"solved! {score_so_far}!")
            print(f"Improved cost: {score_so_far}")
            self.MIN_COST = score_so_far
            return
        moves, home = all_options(board_dict)
        if len(moves) == 0:
            if debug: 
                print(f"no more moves ({n_home}/{total_amph})")
            return
        first = debug
        for amph, options in moves.items(): # every amph that can move
            i, j = amph
            for ti, tj in options:
                new_board = board_dict.copy()
                cost = (abs(ti-i) + abs(tj-j)) * COST[board_dict[amph]]
                new_board[amph] = '.'
                new_board[(ti, tj)] = board_dict[amph]
                self.explore(new_board, debug=first, score_so_far=score_so_far+cost)
                first = False
        return

In [ ]:
Exploration2(inputs, 1500)

In [145]:
print_board(inputs)

#############
#...........#
###A#C#C#D###
###B#D#A#B###
#############


In [139]:
all_options(inputs, True)

(2, 3) can move --> [(1, 1), (1, 2), (1, 4), (1, 6), (1, 8), (1, 10), (1, 11)]
(2, 5) can move --> [(1, 1), (1, 2), (1, 4), (1, 6), (1, 8), (1, 10), (1, 11)]
(2, 7) can move --> [(1, 1), (1, 2), (1, 4), (1, 6), (1, 8), (1, 10), (1, 11)]
(2, 9) can move --> [(1, 1), (1, 2), (1, 4), (1, 6), (1, 8), (1, 10), (1, 11)]

#############
#...........#
###A#C#C#D###
###B#D#A#B###
#############


({(2, 3): [(1, 1), (1, 2), (1, 4), (1, 6), (1, 8), (1, 10), (1, 11)],
  (2, 5): [(1, 1), (1, 2), (1, 4), (1, 6), (1, 8), (1, 10), (1, 11)],
  (2, 7): [(1, 1), (1, 2), (1, 4), (1, 6), (1, 8), (1, 10), (1, 11)],
  (2, 9): [(1, 1), (1, 2), (1, 4), (1, 6), (1, 8), (1, 10), (1, 11)]},
 False)

In [140]:
test_input = """#############
#.A.B.C...D.#
###.#.#C#D###
###.#.#A#B###
#############"""
summary(parse_input(test_input.split('\n')))

(1, 2) can move --> [(3, 3)] HOME!
(1, 4) can move --> [(3, 5)] HOME!
(2, 7) can move --> [(1, 8)]
(2, 9) can move --> [(1, 8)]

#############
#.A.B.C...D.#
###.#.#C#D###
###.#.#A#B###
#############


In [63]:
neighbors((2, 3))

[(3, 3), (2, 4), (1, 3), (2, 2)]

In [56]:
print_board(inputs)

#############
#...........#
###A#C#C#D###
###B#D#A#B###
#############


In [57]:
print_board(inputs_new)

#############
#...........#
###A#C#C#D###
###D#C#B#A###
###D#B#A#C###
###B#D#A#B###
#############


In [ ]:
solve1()

In [ ]:
solve1()

In [ ]:
solve2()

In [ ]:
solve2()